# Instrument Classification - Part 1 : Pre-Processing

We start our first trial by trying to recognise piano (pia), acoustic guitar (gac)

In [36]:
import numpy as np
import librosa
import os
import math
from random import shuffle
from collections import deque
from pydub import AudioSegment
from pydub.utils import make_chunks

In [38]:
dirpia = './Instrument-Classification/IRMAS-TrainingData/pia/'
dirgac = './Instrument-Classification/IRMAS-TrainingData/gac/'
dirgel = './Instrument-Classification/IRMAS-TrainingData/gel/'
dirbuz = './Instrument-Classification/IRMAS-TrainingData/buz/'
dirvio = './Instrument-Classification/IRMAS-TrainingData/vio/'
dirvoi = './Instrument-Classification/IRMAS-TrainingData/voi/'
dirpia_gac = './Instrument-Classification/IRMAS-TrainingData/pia-gac/'
dirpia_gel = './Instrument-Classification/IRMAS-TrainingData/pia-gel/'
dirpia_buz = './Instrument-Classification/IRMAS-TrainingData/pia-buz/'
dirpia_vio = './Instrument-Classification/IRMAS-TrainingData/pia-vio/'
dirpia_voi = './Instrument-Classification/IRMAS-TrainingData/pia-voi/'
dirgac_gel = './Instrument-Classification/IRMAS-TrainingData/gac-gel/'
dirgac_buz = './Instrument-Classification/IRMAS-TrainingData/gac-buz/'
dirgac_vio = './Instrument-Classification/IRMAS-TrainingData/gac-vio/'
dirgac_voi = './Instrument-Classification/IRMAS-TrainingData/gac-voi/'
dirgel_buz = './Instrument-Classification/IRMAS-TrainingData/gel-buz/'
dirgel_vio = './Instrument-Classification/IRMAS-TrainingData/gel-vio/'
dirgel_voi = './Instrument-Classification/IRMAS-TrainingData/gel-voi/'
dirbuz_vio = './Instrument-Classification/IRMAS-TrainingData/buz-vio/'
dirbuz_voi = './Instrument-Classification/IRMAS-TrainingData/buz-voi/'
dirvio_voi = './Instrument-Classification/IRMAS-TrainingData/vio-voi/'

dirnot_pia = './Instrument-Classification/IRMAS-TrainingData/not-piano/'

#instr = ['pia','gac','gel','buz','vio','voi']
instr = ['pia','gac']

#dir = [dirpia,     dirgac,     dirgel,     dirbuz,     dirvio,    dirvoi,
#       dirpia_gac, dirpia_gel, dirpia_buz, dirpia_vio, dirpia_voi,
#       dirgac_gel, dirgac_buz, dirgac_vio, dirgac_voi,
#       dirgel_buz, dirgel_vio, dirgel_voi,
#       dirbuz_vio, dirbuz_voi,
#       dirvio_voi]

dir_array = [dirpia,dirgac,dirpia_gac]

In [39]:
def pre_processing(dir_array):
    x_list = []
    y_list = []
    check = 0
    for dir in dir_array:
        for f in os.listdir(dir):
            try:
                label = dir.split('/')
            
                # downsampled to 22050Hz and downmixed to mono
                wav,sr = librosa.load(dir+f,mono=True,sr=22050)
        
                # normalized by the root mean square energy
                wav = wav / np.sqrt(np.mean(wav**2))
        
                # transformed into mel-spectrograms with given traits
                #mel = librosa.feature.melspectrogram(wav,n_mels=96,fmax=11025,n_fft=1024,hop_length=256,power=1)
                mel = librosa.core.cqt(wav,n_bins=96,hop_length=256)
            
                # decibel scaling
                mel = librosa.core.amplitude_to_db(np.abs(mel))
                
                #just for checking
                if (check%500==0): 
                    print(check," chuncks have been made!")
                check+=1
        
                y_list.append(label[-2])
                x_list.append(mel)
            sexcept:
                print("Empty Track: ",f)
    return (x_list,y_list)

def to_categorical(list):
    list_final = []
    for label in list:
        list_final.append(label.split('-'))
    temp_list = [[0]*len(instr)]*(len(list_final))
    for i in range(len(list_final)):
        x = [0]*len(instr)
        x.insert(0,1)
        x.pop()
        for ins in instr:
            if (ins in list_final[i]): 
                temp_list[i] = [sum(x) for x in zip(temp_list[i],x)]
            x.insert(0,x.pop())
    return temp_list

In [40]:
(x_list,y_list) = pre_processing(dir_array)

x_list = np.expand_dims(np.array(x_list),-1)
y_list = np.array(to_categorical(y_list))

#shuffling sets
id = np.random.permutation(len(x_list))
x_list,y_list = x_list[id], y_list[id]

#splitting to train and test 90/10
split_index = math.floor(len(x_list)*0.90)
x_train = x_list[:split_index]
y_train = y_list[:split_index]
x_test  = x_list[split_index:]
y_test  = y_list[split_index:]

print("Train Size:", x_train.shape)
print("Test  Size:", x_test.shape)

0  chuncks have been made!
500  chuncks have been made!
1000  chuncks have been made!
1500  chuncks have been made!
2000  chuncks have been made!
2500  chuncks have been made!
3000  chuncks have been made!
3500  chuncks have been made!
4000  chuncks have been made!
4500  chuncks have been made!
5000  chuncks have been made!
5500  chuncks have been made!
6000  chuncks have been made!
6500  chuncks have been made!
7000  chuncks have been made!
7500  chuncks have been made!
8000  chuncks have been made!
8500  chuncks have been made!
9000  chuncks have been made!
9500  chuncks have been made!
10000  chuncks have been made!
10500  chuncks have been made!
11000  chuncks have been made!
11500  chuncks have been made!
12000  chuncks have been made!
12500  chuncks have been made!


C:\Users\AgelosK\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


Empty Track:  3_1731_a_n.wav
13000  chuncks have been made!
Empty Track:  3_2026_a_n.wav
Empty Track:  3_2027_a_n.wav
13500  chuncks have been made!
Empty Track:  3_2353_a_n.wav
14000  chuncks have been made!
Empty Track:  3_2701_a_n.wav
14500  chuncks have been made!
15000  chuncks have been made!
15500  chuncks have been made!
16000  chuncks have been made!
16500  chuncks have been made!
17000  chuncks have been made!
17500  chuncks have been made!
18000  chuncks have been made!
18500  chuncks have been made!
19000  chuncks have been made!
19500  chuncks have been made!
20000  chuncks have been made!
20500  chuncks have been made!
21000  chuncks have been made!
21500  chuncks have been made!
22000  chuncks have been made!
22500  chuncks have been made!
23000  chuncks have been made!
23500  chuncks have been made!
24000  chuncks have been made!
24500  chuncks have been made!
25000  chuncks have been made!
25500  chuncks have been made!
26000  chuncks have been made!
26500  chuncks hav

In [41]:
np.save('x_trainPGm_CQT_big.npy',x_train)
np.save('x_testPGm_CQT_big.npy' ,x_test)
np.save('y_trainPGm_CQT_big.npy',y_train)
np.save('y_testPGm_CQT_big.npy' ,y_test)